In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/.shortcut-targets-by-id/18nj7Xok8Hu2jUgOuoiWJPFBZ6v9a-o4A/COVID-19_Radiography_Dataset"

### Important Libraries 

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import glob
import matplotlib.pyplot as plt
import cv2

### Image Preprocessing 

In [ ]:
def pre_processing(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.Canny(image, 80, 100)
    return image

In [ ]:
tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    preprocessing_function= pre_processing
)

### Image Data Generators 

In [ ]:
train=path+"/Train"
val=path+"/Val"
test=path+"/Test"

In [ ]:
gen_train = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2)
gen_val = ImageDataGenerator(rescale = 1./255.)
gen_test = ImageDataGenerator(rescale = 1./255.)

train_gen = gen_train.flow_from_directory(train, batch_size = 64, class_mode = 'categorical', target_size = (224, 224))
validation_gen = gen_val.flow_from_directory(val, batch_size = 64, class_mode = 'categorical', target_size=(224, 224))
test_gen = gen_test.flow_from_directory(test, shuffle = False, batch_size=64, class_mode = 'categorical', target_size=(224, 224))

Found 5786 images belonging to 2 classes.
Found 722 images belonging to 2 classes.
Found 722 images belonging to 2 classes.


### Transfer Learning with VGG19 Base Model



In [ ]:
base_model=tf.keras.applications.VGG19(
    weights="imagenet",
    input_shape=(224,224,3),
    classifier_activation="softmax",
)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512,kernel_initializer='he_uniform'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization()) 
model.add(Dense(2, activation='softmax'))
model.summary()

574710816/574710816 [==============================] - 5s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 512)               512512    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 128)               

In [ ]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)
history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 50, epochs = 15, callbacks = callback)

Epoch 1/15
50/50 [==============================] - 626s 12s/step - loss: 0.6040 - accuracy: 0.7425 - val_loss: 0.6848 - val_accuracy: 0.5000
Epoch 2/15
50/50 [==============================] - 228s 5s/step - loss: 0.5629 - accuracy: 0.7487 - val_loss: 0.6753 - val_accuracy: 0.6025
Epoch 3/15
50/50 [==============================] - 138s 3s/step - loss: 0.5328 - accuracy: 0.7634 - val_loss: 0.6766 - val_accuracy: 0.5000
Epoch 4/15
50/50 [==============================] - 94s 2s/step - loss: 0.5189 - accuracy: 0.7717 - val_loss: 0.6657 - val_accuracy: 0.5097
Epoch 5/15
50/50 [==============================] - 67s 1s/step - loss: 0.4958 - accuracy: 0.7799 - val_loss: 0.6439 - val_accuracy: 0.5360
Epoch 6/15
50/50 [==============================] - 61s 1s/step - loss: 0.4920 - accuracy: 0.7769 - val_loss: 0.6191 - val_accuracy: 0.5789
Epoch 7/15
50/50 [==============================] - 60s 1s/step - loss: 0.4945 - accuracy: 0.7788 - val_loss: 0.6064 - val_accuracy: 0.6150
Epoch 8/15
50/50

In [ ]:
np.save('VGG19-history.npy',history.history)

In [ ]:
# history=np.load('VGG19-history.npy',allow_pickle='TRUE').item()

### Save the Model

In [ ]:
model.save_weights("VGG19.h5")

In [ ]:
model.save("VGG19-Model.h5")

In [ ]:
y_pred = model.predict(test_gen)

12/12 [==============================] - 168s 15s/step


In [ ]:
from google.colab import files
files.download('VGG19-Model.h5') 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('VGG19-history.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>